Importing libraries:
- os - for reading and saving files
- pandas - for working with dataset
- scikit-learn - for machine learning
- string - for removing punctuation
- joblib - for saving model and vectorizer

In [1]:
import os 
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import string 
import joblib

Reading files from folders "astronomia" and "inne" and adding labels from folders name.

In [2]:
def read_files(folder_path, label):
    texts = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            with open(file_path, 'r', encoding='utf-8') as file:
                texts.append(file.read())
    return texts, [label] * len(texts) 

astronomy_folder = 'dane/astronomia'
others_folder = 'dane/inne'

astronomy_texts, astronomy_labels = read_files(astronomy_folder, 'astronomia')
others_texts, others_labels = read_files(others_folder, 'inne')

Creating DataFrame and .csv file.

In [3]:
texts = astronomy_texts + others_texts
labels = astronomy_labels + others_labels

data = {'text': texts, 'label':labels}
df = pd.DataFrame(data)

df.to_csv('dane_zbiorcze.csv', index=False)
print('Dane zapisane.')

Dane zapisane.


In [4]:
df.sample(10)

,text,label
74,"Vindemiatrix (""Epsilon Virginis"") - trzecia co...",astronomia
154,"Liczba stopni swobody, df (ang. ""degrees of fr...",inne
56,Herb Singapuru przedstawia tarczę heraldyczną ...,astronomia
210,William Shakespeare (forma spolszczona: Szeksp...,inne
15,"Obszary H II - obłoki zjonizowanego gazu, głów...",astronomia
101,"Fobos-Grunt (ros. ""Fobos-Grunt"") - rosyjska so...",astronomia
180,Wygląd i symbolika.\nFlaga Antyli Holenderskic...,inne
10,"Jednorożec (łac. ""Monoceros"", dop. ""Monoceroti...",astronomia
71,NGC 4650 A (również PGC 42951) - biegunowa gal...,astronomia
167,Godło Timoru Wschodniego zostało wprowadzone 1...,inne


Cleaning texts from punctuation, polish stopwords and capital letters.

In [5]:
def load_stopwords(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        stopwords = file.read().splitlines()
    return set(stopwords)

stopwords_file = 'polish_stopwords.txt'

polish_stopwords = load_stopwords(stopwords_file)

def preprocess_text(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    words = text.split()
    words = [word for word in words if word not in polish_stopwords]
    return ' '.join(words)

df['text'] = df['text'].apply(preprocess_text)

In [6]:
df.sample(5)

,text,label
139,obserwatorium wiedeńskie obserwatorium astrono...,astronomia
208,dramat satyrowy – starożytna grecka odmiana ko...,inne
47,sts41c ang space transportation system piąta m...,astronomia
17,kepler80 gwiazda położona gwiazdozbiorze łabęd...,astronomia
105,obserwatorium astronomicznometeorologiczne mar...,astronomia


Splitting data into training and testing.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

In [8]:
print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)
print("y_train shape: ", y_train.shape)
print("y_test shape: ", y_test.shape)

X_train shape:  (224,)
X_test shape:  (56,)
y_train shape:  (224,)
y_test shape:  (56,)


Vectorizing the data.

In [9]:
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

Training Naive Bayes model and making prediction. Evaluating the model.

In [10]:
model_NB = MultinomialNB()
model_NB.fit(X_train_vec, y_train)

y_pred_NB = model_NB.predict(X_test_vec)
print('NB Accuracy:', accuracy_score(y_test, y_pred_NB))
print('NB Classification Report:\n', classification_report(y_test, y_pred_NB))

NB Accuracy: 0.8214285714285714
NB Classification Report:
               precision    recall  f1-score   support

  astronomia       0.75      0.96      0.84        28
        inne       0.95      0.68      0.79        28

    accuracy                           0.82        56
   macro avg       0.85      0.82      0.82        56
weighted avg       0.85      0.82      0.82        56



Training Logistic Regression model and making prediction. Evaluating the model.

In [11]:
model_LR = LogisticRegression()
model_LR.fit(X_train_vec, y_train)

y_pred_LR = model_LR.predict(X_test_vec)
print('LR Accuracy:', accuracy_score(y_test, y_pred_LR))
print('LR Classification Report:\n', classification_report(y_test, y_pred_LR))

LR Accuracy: 0.9464285714285714
LR Classification Report:
               precision    recall  f1-score   support

  astronomia       1.00      0.89      0.94        28
        inne       0.90      1.00      0.95        28

    accuracy                           0.95        56
   macro avg       0.95      0.95      0.95        56
weighted avg       0.95      0.95      0.95        56



The Logistic Regression model made predictions better.

In [12]:
y_pred_LR[0:5]

array(['astronomia', 'astronomia', 'inne', 'inne', 'inne'], dtype=object)

In [13]:
X_test.head()

33     korona południowa łac corona australis dop cor...
108    planetozymal planetezymal małe ciało niebieski...
240    mykoryza12 mikoryza345a mycorrhiza – powszechn...
259    hollywood boulevard – ulica hollywood los ange...
154    liczba stopni swobody df ang degrees of freedo...
Name: text, dtype: object

Saving vectorizer and model.

In [14]:
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')
joblib.dump(model_LR, 'logistic_regression_model.pkl')

print('Pliki zostały zapisane.')

Pliki zostały zapisane.


Validating the model on new data from other sources than "wikipedia.pl".

In [26]:
val_df = pd.read_csv('dane_walidacyjne.csv', encoding='utf-8', sep=';')
val_df_clean = val_df.copy()
val_df_clean['text'] = val_df['text'].apply(preprocess_text)
val_df_clean.head()

,text,label
0,1992 astronomowie podejrzewali posiadali jedno...,astronomia
1,1000 puzzle delle formiche „mrówkowe” obrazek ...,inne
2,c2020 f3 neowise – kometa długookresowa odkryt...,astronomia
3,planety powinny fotografowanie możliwie najwyż...,astronomia
4,niewidoma marielaure mieszka paryżu okolicy mu...,inne


In [27]:
val_df_clean.shape

(20, 2)

In [28]:
X_val = val_df_clean['text']
y_val = val_df_clean['label']
X_val_vec = vectorizer.transform(X_val)

y_val_pred_LR = model_LR.predict(X_val_vec)
print('LR Accuracy:', accuracy_score(y_val, y_val_pred_LR))
print('LR Classification Report:\n', classification_report(y_val, y_val_pred_LR))

LR Accuracy: 0.7
LR Classification Report:
               precision    recall  f1-score   support

  astronomia       1.00      0.40      0.57        10
        inne       0.62      1.00      0.77        10

    accuracy                           0.70        20
   macro avg       0.81      0.70      0.67        20
weighted avg       0.81      0.70      0.67        20



In [31]:
y_val_pred_df = pd.DataFrame(y_val_pred_LR, columns=['predicted_label'])
val_pred_df = pd.concat([val_df['text'], val_df['label'], y_val_pred_df], axis=1)
val_pred_df

,text,label,predicted_label
0,"Do roku 1992 astronomowie podejrzewali, ale ni...",astronomia,astronomia
1,1000 Puzzle delle Formiche czyli „mrówkowe” Ob...,inne,inne
2,"C/2020 F3 (NEOWISE) – kometa długookresowa, od...",astronomia,astronomia
3,Planety powinny być fotografowanie możliwie ja...,astronomia,inne
4,Niewidoma Marie-Laure mieszka w Paryżu w okoli...,inne,inne
5,Bieszczady miały być dla Wiktora Forsta azylem...,inne,inne
6,Nowa gazetka już dostępna!\nZ radością informu...,inne,inne
7,Nie każdy może lub chce zabrać swój teleskop i...,astronomia,inne
8,Księżyc w fazie 3 doby i 7 godzin po nowiu. Ba...,astronomia,inne
9,Wentworth 1000 „Soulmates”\nPrzecudowne puzzle...,inne,inne


The model cannot predict based on data that has a source other than "wikipedia.pl". The set of training data should be increased by data from other sources.